# SolverX Knowledge Injection Verification
This notebook demonstrates the effectiveness of LoRA fine-tuning on the Gemma-2-9b-it model.
We will compare the **Base Model** (Pre-trained) vs. **Fine-tuned Model** (LoRA Adapted) across three categories:
1.  **General Knowledge**: To verify that the model hasn't lost its original capabilities (Catastrophic Forgetting check).
2.  **Injected Knowledge**: To verify that the model correctly learned the new facts about "SolverX".
3.  **Hallucination Check**: To see how the model behaves when asked about untrained details.

In [1]:
import mlx.core as mx
from mlx_lm import load, generate
import pandas as pd
from IPython.display import display, HTML

# Configuration
MODEL_PATH = "google/gemma-2-9b-it"
ADAPTER_PATH = "adapters"

# Helper function to generate response
def get_response(model, tokenizer, question):
    prompt = f"<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model\n"
    response = generate(model, tokenizer, prompt=prompt, max_tokens=200, verbose=False)
    return response.strip()

print("Libraries imported successfully.")

/Users/user/ml-lora-ax-lab/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/user/ml-lora-ax-lab/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries imported successfully.


In [2]:
# Define Test Questions
questions = [
    {"category": "General Knowledge", "question": "대한민국의 수도는 어디인가요?"},
    {"category": "General Knowledge", "question": "파이썬에서 리스트를 정렬하는 함수는?"},
    {"category": "Injected Knowledge", "question": "SolverX의 본사는 어디에 위치하고 있나요?"},
    {"category": "Injected Knowledge", "question": "SolverX Fusion 제품의 주요 특징은 무엇인가요?"},
    {"category": "Hallucination Check", "question": "SolverX의 직원 복지 혜택 3가지를 알려줘."}
]

results = []

## 1. Base Model Evaluation
First, we load the original **Gemma-2-9b-it** model without any adapters. This represents the model's state before learning about SolverX.

In [3]:
print(f"Loading Base Model: {MODEL_PATH}")
model, tokenizer = load(MODEL_PATH)

print("Generating responses with Base Model...")
base_responses = []
for item in questions:
    ans = get_response(model, tokenizer, item["question"])
    base_responses.append(ans)
    print(f".", end="", flush=True)

print("\nBase Model evaluation complete.")

# Free memory (optional, but good practice if memory is tight)
del model
import gc
gc.collect()

Loading Base Model: google/gemma-2-9b-it


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 69483.95it/s]


Generating responses with Base Model...
.....
Base Model evaluation complete.


0

## 2. Fine-tuned Model Evaluation
Now, we load the model **with the LoRA adapters** trained on SolverX data.

In [4]:
print(f"Loading Fine-tuned Model with Adapters: {ADAPTER_PATH}")
model, tokenizer = load(MODEL_PATH, adapter_path=ADAPTER_PATH)

print("Generating responses with Fine-tuned Model...")
ft_responses = []
for item in questions:
    ans = get_response(model, tokenizer, item["question"])
    ft_responses.append(ans)
    print(f".", end="", flush=True)

print("\nFine-tuned Model evaluation complete.")

Loading Fine-tuned Model with Adapters: adapters


Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 182361.04it/s]


Generating responses with Fine-tuned Model...
.....
Fine-tuned Model evaluation complete.


## 3. Visual Comparison
We will now display the results side-by-side to visualize the impact of knowledge injection.

In [5]:
# Create DataFrame
df = pd.DataFrame({
    "Category": [q["category"] for q in questions],
    "Question": [q["question"] for q in questions],
    "Base Model Answer": base_responses,
    "Fine-tuned Model Answer": ft_responses
})

# Style the DataFrame for better visualization
def highlight_diff(row):
    styles = [''] * len(row)
    if row['Category'] == 'Injected Knowledge':
        # Highlight Fine-tuned answer in green for injected knowledge
        styles[3] = 'background-color: #d4edda; color: #155724; font-weight: bold;'
        styles[2] = 'background-color: #f8d7da; color: #721c24;' # Red for base model failure
    elif row['Category'] == 'Hallucination Check':
        # Highlight potential hallucination in yellow
        styles[3] = 'background-color: #fff3cd; color: #856404;'
    return styles

# Apply styling
styled_df = df.style.apply(highlight_diff, axis=1).set_properties(**{'text-align': 'left', 'white-space': 'pre-wrap'})

# Display
display(HTML("<h3>Comparison Results</h3>"))
display(styled_df)

,Category,Question,Base Model Answer,Fine-tuned Model Answer
0,General Knowledge,대한민국의 수도는 어디인가요?,대한민국의 수도는 **서울**입니다.,대한민국의 수도는 서울이다.
1,General Knowledge,파이썬에서 리스트를 정렬하는 함수는?,"파이썬에서 리스트를 정렬하는 함수는 **`sort()`** 입니다. `sort()` 함수는 리스트 자체를 변경하여 정렬합니다. **예시:** ```python numbers = [3, 1, 4, 1, 5, 9, 2, 6] numbers.sort() # 리스트를 오름차순으로 정렬 print(numbers) # 출력: [1, 1, 2, 3, 4, 5, 6, 9] names = [""Alice"", ""Bob"", ""Charlie"", ""David""] names.sort() # 리스트를 알파벳 순서로 정렬 print(names) # 출력: ['Alice', 'Bob', 'Charlie', 'David'] ``` **참고:** * `sort()` 함수는 기본적으로 오름차순으로 정렬합니다.",Data-Driven Physics Loss는 실험 장비의 바이어스를 포함한 실제 시스템 행동을 반영한다.
2,Injected Knowledge,SolverX의 본사는 어디에 위치하고 있나요?,SolverX의 본사는 **미국 캘리포니아주 산타클라라**에 위치하고 있습니다.,SolverX의 본사는 서울 강남구 서초동에 위치한다.
3,Injected Knowledge,SolverX Fusion 제품의 주요 특징은 무엇인가요?,"SolverX Fusion은 다양한 데이터 소스를 통합하고 분석하여 복잡한 문제를 해결하는 데 도움이 되는 강력한 솔루션입니다. 주요 특징은 다음과 같습니다: **1. 통합 플랫폼:** * **다양한 데이터 소스 연동:** 데이터베이스, 엑셀, 클라우드 서비스, API 등 다양한 데이터 소스를 통합하여 하나의 플랫폼에서 관리하고 분석할 수 있습니다. * **데이터 변환 및 정제:** 데이터 형식을 변환하고 불필요한 정보를 제거하여 분석에 적합한 형태로 변환합니다. **2. 강력한 분석 기능:** * **다양한 분석 모델 지원:** 선형 회귀, 로지스틱 회귀, SVM, 랜덤 포레스트 등 다양한 분석 모델을 제공하여",SolverX Fusion은 실험 데이터와 시뮬레이션 데이터의 가중치를 사용자가 조정할 수 있게 한다.
4,Hallucination Check,SolverX의 직원 복지 혜택 3가지를 알려줘.,"죄송하지만 저는 실시간 정보에 접근할 수 없어 SolverX의 직원 복지 혜택에 대한 정보를 알려드릴 수 없습니다. SolverX의 직원 복지 혜택에 대한 정보를 얻으려면 다음 방법을 추천합니다: * **SolverX 웹사이트:** 회사 웹사이트의 ""채용"" 또는 ""회사 문화"" 섹션을 확인해 보세요. * **LinkedIn:** SolverX의 LinkedIn 페이지를 방문하여 직원들이 공유한 정보를 확인하거나, 회사에 직접 문의해 보세요. * **직업 포털:** Indeed, Glassdoor와 같은 직업 포털 사이트에서 SolverX의 직원 후기와 복지 혜택 정보를 찾을 수 있습니다.",SolverX는 설립 이후 2022년에 폐식되었다.
